<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/Emotionline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emitionline 데이터셋

In [1]:
! tar -xvzf /content/drive/My\ Drive/Datasets/Emotionlines/EmotionLines_friends_annotation.tar.gz

EmotionLines/Friends/friends_train.json
EmotionLines/Friends/friends_dev.json
EmotionLines/Friends/friends_test.json
EmotionLines/README


In [2]:
import pandas as pd
import json

In [3]:
with open('/content/EmotionLines/Friends/friends_train.json', encoding='utf-8', mode='r') as f:
  array = json.load(f)

data =[]
for i in range(len(array)):
  data += array[i]
print(f"Total length of data : {len(data)}")
df = pd.DataFrame(data)

Total length of data : 10561


In [4]:
df

,speaker,utterance,emotion,annotation
0,Chandler,also I was the point person on my companys tr...,neutral,4100000
1,The Interviewer,You mustve had your hands full.,neutral,5000000
2,Chandler,That I did. That I did.,neutral,5000000
3,The Interviewer,So lets talk a little bit about your duties.,neutral,5000000
4,Chandler,My duties? All right.,surprise,2000030
...,...,...,...,...
10556,Chandler,You or me?,neutral,3000011
10557,Ross,"I got it. Uh, Joey, women don't have Adam's ap...",non-neutral,2100011
10558,Joey,"You guys are messing with me, right?",surprise,0000050
10559,All,Yeah.,neutral,4000010


In [5]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [6]:
# change the value to dtype: Category
df['emotion'] = pd.Categorical(df['emotion'])

# cat(egory).codes print the code of category
emotion_codes = df.emotion.cat.codes

In [7]:
emotion_codes

0        4
1        4
2        4
3        4
4        7
        ..
10556    4
10557    5
10558    7
10559    4
10560    5
Length: 10561, dtype: int8

In [8]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [9]:
for i, utter in enumerate(df.utterance):
  if i < 5:
    print(utter)
  else:
    break

also I was the point person on my companys transition from the KL-5 to GR-6 system.
You mustve had your hands full.
That I did. That I did.
So lets talk a little bit about your duties.
My duties?  All right.


# make data



make the dataset to tensor

In [10]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

In [11]:
x_train = df.pop('utterance')
y_train = emotion_codes

In [12]:
x_train.values

array(['also I was the point person on my company\x92s transition from the KL-5 to GR-6 system.',
       'You must\x92ve had your hands full.', 'That I did. That I did.',
       ..., 'You guys are messing with me, right?', 'Yeah.',
       'That was a good one. For a second there, I was like, "whoa."'],
      dtype=object)

In [13]:
y_onehot = tf.one_hot(y_train, depth=8)
y_onehot.shape

TensorShape([10561, 8])

이 이하는 실행 안 해도 됨

In [14]:
dataset = tf.data.Dataset.from_tensor_slices(x_train)

In [15]:
for t in dataset.take(5):
  print(t)

tf.Tensor(b'also I was the point person on my company\xc2\x92s transition from the KL-5 to GR-6 system.', shape=(), dtype=string)
tf.Tensor(b'You must\xc2\x92ve had your hands full.', shape=(), dtype=string)
tf.Tensor(b'That I did. That I did.', shape=(), dtype=string)
tf.Tensor(b'So let\xc2\x92s talk a little bit about your duties.', shape=(), dtype=string)
tf.Tensor(b'My duties?  All right.', shape=(), dtype=string)


In [16]:
train_data = dataset.shuffle(len(df))
for t in train_data.take(5):
  print(t)

tf.Tensor(b"Okay, now you're even cuter!!", shape=(), dtype=string)
tf.Tensor(b'I\xc2\x92m not freaking out.', shape=(), dtype=string)
tf.Tensor(b'Oh yeah, he looked like a real lumberjack in those pink lacys.', shape=(), dtype=string)
tf.Tensor(b'Oh my God! I\xc2\x92m gonna have to find another minister.', shape=(), dtype=string)
tf.Tensor(b'Listen, I\xc2\x92ve got a secret for ya. I let him win.', shape=(), dtype=string)


# Tokenizer

In [57]:
tokenizer = K.preprocessing.text.Tokenizer(num_words = 4000, char_level=False)

In [58]:
tokenizer.fit_on_texts(x_train.values)

In [59]:
x_in = tokenizer.texts_to_sequences(x_train)

In [60]:
x_in = tf.keras.preprocessing.sequence.pad_sequences(x_in)
print(x_in)

[[   0    0    0 ... 2888  905  906]
 [   0    0    0 ...   40  709  827]
 [   0    0    0 ...    9    1   73]
 ...
 [   0    0    0 ...   36   14   34]
 [   0    0    0 ...    0    0   17]
 [   0    0    0 ...   35   42  177]]


In [61]:
x_in.shape

(10561, 68)

# Embedding

In [62]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, ReLU, Softmax

## Hyperparameter

In [63]:
INPUT_DIM = 4000
EMBED_DIM = 1000
INPUT_LEN = 68
RNN_DEPTH = 128
FIRST_DENSE = 64
OUTPUT_LEN = 8
BATCH_SIZE = 100
EPOCHS = 10
SHUFFLE = True

Model Create

In [64]:
model = K.Sequential()
model.add(Embedding(input_dim = INPUT_DIM, output_dim=EMBED_DIM, input_length=INPUT_LEN))
model.add(Flatten())
model.add(Dense(FIRST_DENSE))
model.add(ReLU())
model.add(Dense(OUTPUT_LEN))
model.add(Softmax())

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 68, 1000)          4000000   
_________________________________________________________________
flatten_5 (Flatten)          (None, 68000)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4352064   
_________________________________________________________________
re_lu_5 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 520       
_________________________________________________________________
softmax_5 (Softmax)          (None, 8)                 0         
Total params: 8,352,584
Trainable params: 8,352,584
Non-trainable params: 0
____________________________________________

In [65]:
model.compile(optimizer="Adam", loss=K.losses.CategoricalCrossentropy())

In [66]:
model.fit(x=x_in, y=y_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=SHUFFLE)

ValueError: ignored

# Model evaluation

In [27]:
def make_test_sample(text):
  test_data = tokenizer.texts_to_sequences(text)
  test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=68)
  print(test_data)
  print(test_data.shape)
  return test_data

In [28]:
test_data = make_test_sample(["Hello Hello I'm Jisoo"])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0 191 191  52]]
(1, 68)


In [29]:
result = model.predict(test_data, batch_size=BATCH_SIZE)

In [30]:
answer = result.argmax()
print(emo_cat.categories[answer])

NameError: ignored

# Categories And codes

In [31]:
emo_cat = pd.Categorical(df['emotion'])
emo_cat.categories
emo_cat.codes

array([4, 4, 4, ..., 7, 4, 5], dtype=int8)

In [32]:
emo_df = pd.DataFrame(
    {"emotion" : emo_cat.categories}
)
emo_df

,emotion
0,anger
1,disgust
2,fear
3,joy
4,neutral
5,non-neutral
6,sadness
7,surprise


# Make RNN model

In [33]:
from tensorflow.keras.layers import SimpleRNN

In [42]:
rnnModel = K.Sequential()
rnnModel.add(Embedding(input_dim = INPUT_DIM, output_dim=EMBED_DIM, input_length=INPUT_LEN))
rnnModel.add(SimpleRNN(RNN_DEPTH))
# rnnModel.add(SimpleRNN(RNN_DEPTH))
rnnModel.add(Flatten())
rnnModel.add(Dense(FIRST_DENSE))
rnnModel.add(ReLU())
rnnModel.add(Dense(OUTPUT_LEN))
rnnModel.add(Softmax())

rnnModel.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 68, 1000)          4000000   
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 128)               144512    
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
re_lu_3 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 520       
_________________________________________________________________
softmax_3 (Softmax)          (None, 8)                

In [43]:
rnnModel.compile(optimizer="Adam", loss=K.losses.CategoricalCrossentropy())

In [44]:
rnnModel.fit(x=x_in, y=y_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=SHUFFLE)

Epoch 1/10
106/106 [==============================] - 28s 267ms/step - loss: 1.5767
Epoch 2/10
106/106 [==============================] - 28s 265ms/step - loss: 1.2504
Epoch 3/10
106/106 [==============================] - 28s 265ms/step - loss: 0.8479
Epoch 4/10
106/106 [==============================] - 27s 259ms/step - loss: 0.5965
Epoch 5/10
106/106 [==============================] - 28s 261ms/step - loss: 0.5373
Epoch 6/10
106/106 [==============================] - 27s 259ms/step - loss: 0.4176
Epoch 7/10
106/106 [==============================] - 26s 247ms/step - loss: 0.3568
Epoch 8/10
106/106 [==============================] - 26s 246ms/step - loss: 0.3205
Epoch 9/10
106/106 [==============================] - 26s 244ms/step - loss: 0.2992
Epoch 10/10
106/106 [==============================] - 26s 247ms/step - loss: 0.2813


# Make Validation Generator

In [53]:
def validation_preprocessor(tokenizer):
  with open('/content/EmotionLines/Friends/friends_dev.json', encoding='utf-8', mode='r') as f:
    array = json.load(f)

  data =[]
  for i in range(len(array)):
    data += array[i]
  print(f"Total length of data : {len(data)}")
  df = pd.DataFrame(data)

    # change the value to dtype: Category
  df['emotion'] = pd.Categorical(df['emotion'])

  # cat(egory).codes print the code of category
  emotion_codes = df.emotion.cat.codes

  x_train = df.pop('utterance')
  y_train = emotion_codes
  y_onehot = tf.one_hot(y_train, depth=8)

  tokenizer.fit_on_texts(x_train.values)
  x_in = tokenizer.texts_to_sequences(x_train)
  x_in = tf.keras.preprocessing.sequence.pad_sequences(x_in)

  return x_in, y_onehot

In [49]:
x_in, y_onehot = validation_preprocessor(tokenizer)

Total length of data : 1178
       speaker  ... annotation
0       Phoebe  ...    0002120
1       Monica  ...    1000130
2         Ross  ...    3000200
3     Chandler  ...    0500000
4         Joey  ...    0040100
...        ...  ...        ...
1173    Monica  ...    2030000
1174    Rachel  ...    0040010
1175    Monica  ...    0050000
1176    Rachel  ...    0050000
1177    Monica  ...    0040010

[1178 rows x 4 columns]


In [50]:
rnnModel.evaluate(x=x_in, y = y_onehot)

37/37 [==============================] - 1s 14ms/step - loss: 3.6107


3.6106910705566406

In [52]:
model.evaluate(x=x_in, y = y_onehot)

ValueError: ignored